# Snowflake : Model Agnostic Acc. for DCNN https://arxiv.org/pdf/1708.02579.pdf

## Import

Example of journalling along with the codebase.

In [57]:
import numpy as np
import pandas as pd
import math

In [29]:
#display(data.describe())
from IPython.display import display # Allows the use of display() for DataFrames


In [28]:
#print("Number of missing values", data.isnull().sum().sum())


## Abstract

Hardware used : Xilinx Zynq XC7Z045 SoC. 

Claims:

1) Only system capable of achieve 91% efficiency across different modern Arch

2) First system with GoogleNet and ResNet as benchmark

In [7]:
theor_peak_throput = 128 * pow(10,9) #ops/s

In [11]:
empirical_peak_throput = {}

In [12]:
empirical_peak_throput["Alex"] = 120 * pow(10,9) #ops/s on Alexnet = 100Frames/second

In [14]:
empirical_peak_throput["GoogleNet"] = 116 * pow(10,9) #ops/s on Googlenet = 36 Frames/second

In [15]:
empirical_peak_throput["ResNet50"] = 122 * pow(10,9) #ops/s on ResNet50 = 17 Frames/second

## Introduction

2 main challenges for CNN accelerators are : computation and bandwidth. This paper deals primarily with computation. I would add area as another vector. 

Measures of Success:

1. Computational efficiency = measured performance/peak performance
2. Computational efficiency = Actual Frames Processed(per sec)/Theoritical Max. that can be processed(per sec)

Opportunities

Computation Complexity is high but there is inherent parallelism

Challenges: 

Highly varied data access patterns of CNNs => an accelerator 
1. designed for one CNN architecture will perform poorly in another
2. performs well for some layers but poorly for other

Additional Introduction Notes: 

This paper looks at convolitional + max pool + nonlinear layers. Not at FC layers. 
FC layer's constraints are more in bandwidth than computation. They need faster memory and compression techniques

## Related Work

Eyeriss : 

12 X 14 grid of PEs for CNN
Data Fed from 108KB Scratchpad buffer
Performance figures in 2 contexts:
1. Including DRAM load latency
2. Excluding DRAM load latency - Assuming that DRAM latency is easy to optimize

In [21]:
Eyeriss = {}
Eyeriss["rowsize"] = 12
Eyeriss["colsize"] = 14
Eyeriss["scratchpad"] = 108 * pow(2,10)
Eyeriss["comp_eff_accounting_dram_lat"] = 68.8 # percent
Eyeriss["comp_eff_notaccounting_dram_lat"] = 76.7 # percent

DianNao:

Difficulties with DianNao:
1. Benchmarks with selected layers instead of entire n/ws


In [23]:
DianNao = {}
DianNao["comp_eff_layer_normal"] = 84.8
DianNao["comp_eff_layer_peak"] = 90.8

ShiDianNao:

Improved DianNao but below problems persist:
1. With small CNNs layers with tens of kilobytes of weights and maps data
3. Tens to hundreds of million operations per frame

Snowflake does:
1. N/Ws with several megabytes of weights and maps data
2. Billions of operations per frame

In [26]:
Caffeine = {}
SongYaoetal = {}
Caffeine["comp_eff"] = 73.3
SongYaoetal["comp_eff"] = 80.3

## CNN Overview

### Setting the stage

<img src="images/CNN one layer.png" alt="" style="width: 50%"/>

Taxonomy: 

iC => No. of Input Maps/Channels(R,G,B = 3 for first layer which are input image)
iW => Width of each input Map
iH => Height of each input Map
Input Dimensions => iC X iW X iH

oC => No. of Output Maps/Channels
oW => Width of each output Map
oH => Height of each output Map
Output Dimensions => oC X oW X oH

kW => Width of Kernels/Filters
kH => Hight of Kernel/Filters
Number of Kernels => oC
Each Kernel's Dimensions => iC X kW X kH

Weight => Each Element of Kernel 

Algorithm per Layer is in below Cell

In [36]:
iW = 0; iH = 0 ; iC = 0 
oW = 0; oH = 0 ; oC = 0
kW = 0; kH = 0 

Downsampling/MaxPooling: 

Max operation over PXP window and produces single output
Stride: Maxpooling is done by striding(sliding/moving) by s on right and bottom to get the new window to Maxpool

In [39]:
mIW = 0; mIH = 0; #input width,height for maxpool layer
# mOW, mOH => Output width

In [40]:
maxpool_window_width = 2;
stride = 2;
mOW = mIW/pow(stride,2)

Non Linearity/Activation Function

Softmax Layer/FC Layer/ 1X1 Convolution
1. Convert all inputs to a single vector.
2. Apply a matrix of weights to produce output vector

Implementation Notes : 

FC layers are more bandwidth dependant. Need faster memory and compression techniques to handle these

Computational challenge is less of a challenge

### Charecteristics of CNN models:

3 types of layers when seen thro' computational eyes:
1. Conventional : As seen in first example. Alexnet + VGG used these only
2. Inception Modules : Has different sized kernels for same layer
3. Residual Modules : Complicated multiple layer jumping and math with large number of input maps

## Data Organization

Primary latency hiding technique : Organize data in form of traces. 

Trace is a contiguous region in memory that an instruction performs an operation on. 

Motivation is to hide latencies of non-compute instructions behind compute cycles. By hiding latencies of branches, true dependencies and memory load and stores helps in achieving peak performance. 

Nature of inputs is 3 dimensional and hence gives 3 ways to iterate on the data. 

For kernel, width and height are small  Range <11 But the depth is very high Range 32-2048 (Resnet)

Data organized in depth-minor format results in longest trace.


In [45]:
iC = 256; kW =3 ; kH = 3 
trace_length = iC *kW
trace_length

768

if trace length is 768 and a single PE is doing this math for 768 cycles. Once PEs are assigned such traces, they are busy for very long time and controller can do other operations in this period of time.

Instructions operating on a trace are called vector instructions.


## MicroArchitecture

Primarily accelerating MAC. Other operartions : ReLU, maxpooling.
Sigmoid and hyperbolic tangent are added as piece-wise linear models

### Control

Similar to five stage RISC

##### FETCH

No loop unrolling used as Snowflake is designed to be good at hiding latencies. Longest branch was to an instruction within 512 instructions from branch. 

In [50]:
instruction_width = 32

In [53]:
min_cache_size = 2 * pow(2,10) #Why? Understand theory or ask Aram

In [54]:
cache_size = 2*min_cache_size #Double buffered so as to overlap latency of fetching next instruction stream

#### Decode:

Checks for data dependancies and stalls for fetch of further instructions until dependant instruction commits 

#### Instruction Despatch

1. Reads source operands from RF.
2. Keeps track of number of loads issued to on-chip buffers of Compute core. Important to prevent vector instruction from reading data from buffers while load is pending. Otherwise read-after-write hazards will happen


#### ALU

1. Only for bookkeeping and kept simple supporting a subset of operations of other ALU
2. Eg. : Adder is used to increment address of compute core's bufferss from where next trace is to be read

#### RF

Dual ported 32 x 32-bit registers
Upto 2 source operands are read from RF in dispatch stage

In [63]:
control_RF ={}
control_RF["ports"] = 2
control_RF["wordlength"] = 32
control_RF["depth"] =32
control_RF["address_bit_width"] = math.log(control_RF["depth"],2)


In [64]:
control_RF["address_bit_width"]

5.0

### Compute Core

Contains multiple compute clusters and has hierarchical structure. 

<img src="images/SFComputeCore.png" alt="" style="width: 70%"/>


#### Compute core Math Model

In [67]:
vMAC= {}
vMAC["cnt_MAC"] = 16

In [68]:
cunit = {}
cunit["cnt_vMAC"] = 4
cunit["cnt_vMAX"] = 1 
cunit["cnt_tracedecode"] = 1
cunit["maps_buffer"] =1

In [70]:
compute_cluster = {}
compute_cluster["cnt_cunits"] = 4

#### Trace Decoders

##### Trace Decoders

Vector Instruction Path

Control Core -> FIFO queue -> Trace Decoder

Instruction Contents : 
1. Start Address of trace in maps buffer
2. Length of trace
3. ID of consumer trace decoder

Once instruction is recieved by a decoder:
1. Decoder increments start address
2. Requests a line from Map Buffer. One line per cycle till end of length of trace
3. After this is done, fetch next trace instruction. Control core takes care of this instruction's availability

##### MAC Trace Decoder

1. Fetch cache lines from map buffer and sends to vMACs
2. Provides address to index into weight buffers inside each vMAC
3. Responsible for ordering vMACs to output results accumulated in thier accum. register. Signal is sent along with last address of final trace of computation

##### MAX trace Decoder

1. Fetch cache lines from map buffer and sends to vMACs
2. Responsible for ordering vMAXs to output results. Signal is sent along with last address of final trace of computation

##### Trace Move Decoder

1. Movement from  maps buffer of one CU to another CU (CU trace move). Destination should be in same cluster.
2. store a trace to memory (memory trace move)

Both functions above share the access to maps buffer

##### Strength due to this model

All trace decoders continue processing their vector instructions while control core executes scalar instructions to prepare next set of vector instructions.

Trace Decoders enable SF to execute & commit vector instructions OOO wrt scalar instructions.Correctness is guaranteed because scalar instruction never reads data produced by vector instruction.  Due to this control core does'nt need speculative execution.


#### MAC

In [72]:
operand_bit_width = 16
accum_bit_width   = 32

Second operand of Adder could be 
1. it's own output
2. Partially Accumulated result
3. Activation from previous layer

<img src="images/vMac parallelism.png" alt="" style="width: 100%"/>


Coordinates in gray : Constant across Macs

Coordinates in black : Varying across Macs

## Dig Deeper

1. Deep Compressed NN
2. Residual Learning

In [55]:
%pwd

u'/Users/rv939730/Learn/Hardware/Papers_Math_and_Summary'